In [1]:
import numpy as np
import pandas as pd
import os
for dirname, _, filenames in os.walk('/kaggle/input/cord-19-eda-parse-json-and-generate-clean-csv'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [2]:
from fastai.text import * 

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
from langdetect import detect

In [ ]:
!pip install googletrans

In [ ]:
from googletrans import Translator

[[](http://)](http://)

* https://www.kaggle.com/xhlulu/cord-19-eda-parse-json-and-generate-clean-csv
* https://www.kaggle.com/danielwolffram/topic-modeling-finding-related-articles



First we load the data inside each of the csv files.


In [ ]:
path = '/kaggle/input/cord-19-eda-parse-json-and-generate-clean-csv/'

In [ ]:
pmc_custom_license_df = pd.read_csv(path + "clean_pmc.csv")
noncomm_use_df = pd.read_csv(path + "clean_noncomm_use.csv")
comm_use_df = pd.read_csv(path + "clean_comm_use.csv")
biorxiv = pd.read_csv(path + "biorxiv_clean.csv")

In [ ]:
data = pd.concat([pmc_custom_license_df, noncomm_use_df, comm_use_df, biorxiv], axis =0)

We subset the dataset to have the paper id and text of the document only

In [ ]:
covid_papers = data[data.text.str.contains('COVID-19|SARS-CoV-2|2019-nCov|SARS Coronavirus 2|2019 Novel Coronavirus')][['paper_id', 'text']]

In [ ]:
data = covid_papers

To detect the language of the papers we use the function detect from the langdetect library

In [ ]:
data['language'] = data['text'].map(detect)

In [ ]:
data.head()

In [ ]:
data.shape

In [ ]:
data['language'].value_counts()

Four papers are written in Spanish.

In [ ]:
indices_to_translate = data[data['language']=='es']['text'].index

In [ ]:
translator = Translator()

In [ ]:
for i in indices_to_translate:

    if (len(data.loc[i, 'text']) > 3900):
        data.loc[i, 'text'] = data.loc[i, 'text'][:3900]
    paper = translator.translate(data.loc[i, 'text'])
    data.loc[i, 'text'] = paper.text


In [ ]:
data.loc[indices_to_translate, 'text']

The papers are now translated

In [ ]:
data.to_csv('data_covid19_papers.csv')

# Language model data



* Training on 90% of the data and validating on 10%
* The function TextLMDataBunch creates a TextDataBunch suitable for training a language model.


In [ ]:
train, validation = train_test_split(data, test_size=0.1, random_state=42, shuffle= True)

In [ ]:
data_lm = TextLMDataBunch.from_df('.',  train, validation,
                                  label_cols='text')

Saving the results to save time in the future.

In [ ]:
data_lm.save('data_lm_export.pkl')

In [ ]:
data_lm = load_data('.', 'data_lm_export.pkl')

Now making the language model learn. It has already learnt some features of basic English using Wikipedia and now I am training it to learn the peculiarities of Covid-19 papers and articles.

In [ ]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

Training for 4 epochs

In [ ]:
learn.fit_one_cycle(4, 2e-2) 

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(10, slice(1e-4, 1e-3))

In [ ]:
learn.save_encoder('ft_enc')


# Inference


In [ ]:
np.random.seed(42)

In [ ]:
learn.predict("risks factors", n_words=100, temperature=0.1)

In [ ]:
learn.predict("smoking", n_words=100, no_unk=True, temperature=0.1)

In [ ]:
learn.predict("pre-existing pulmonary disease", n_words=50, no_unk=True, temperature=0.05)

In [ ]:
learn.predict("Co-infections", n_words=50, no_unk=True, temperature=0.1)

In [ ]:
learn.predict("comorbidities", n_words=50, no_unk=True, temperature=0.1)

In [ ]:
learn.predict("Neonates", n_words=50, no_unk=True, temperature=0.1) 

In [ ]:
learn.predict("pregnant women", n_words=50, no_unk=True, temperature=0.1)

In [ ]:
learn.predict("Socio-economic factors", n_words=100, no_unk=True, temperature=0.1) 

In [ ]:
learn.predict("behavioral factors", n_words=80, no_unk=True, temperature=0.1)

In [ ]:
learn.predict("Transmission dynamics of the virus", n_words=50, no_unk=True, temperature=0.1) 

In [ ]:
learn.predict("basic reproductive number", n_words=80, no_unk=True, temperature=0.1)

In [ ]:
learn.predict("incubation period", n_words=50, no_unk=True, temperature=0.1)

In [ ]:
learn.predict("serial interval", n_words=50, no_unk=True, temperature=0.1) 

In [ ]:
learn.predict("environmental factors", n_words=50, no_unk=True, temperature=0.1) 

In [ ]:
learn.predict("modes of transmission", n_words=50, no_unk=True, temperature=0.1) 

In [ ]:
learn.predict("Severity of disease", n_words=50, no_unk=True, temperature=0.1)

In [ ]:
learn.predict("risk of fatality among symptomatic hospitalized patients", n_words=50, no_unk=True, temperature=0.1)

In [ ]:
learn.predict("high-risk patient", n_words=50, no_unk=True, temperature=0.1)

In [ ]:
learn.predict("Susceptibility of populations", n_words=50, no_unk=True, temperature=0.1)

In [ ]:
learn.predict("Public health mitigation measures", n_words=50, no_unk=True, temperature=0.1) 

In [ ]:
learn.predict("What do we know about COVID-19 risk factors?", n_words=100, no_unk=True, temperature=0.1)

In [ ]:
data.head()